In [157]:
import pandas as pd
import numpy as np
import os
import keras
from datetime import timedelta

In [167]:
#os.chdir("src") used to reset to original working directory

In [158]:
model = keras.Sequential()
model.add(keras.layers.LSTM(8,input_shape= (3,1)))
model.add(keras.layers.Dense(1))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 8)                 320       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [168]:
os.chdir("../") #Used to go out into the cmpe-351-group-1 folder to access the data without changing directories multiple times

In [169]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [170]:
crypto_dict = {
    "bitcoin":"btc",
    "avalanche":"avax",
    "doge":"doge",
    "ethereum":"eth",
    "solana":"sol"
}

In [171]:
def get_dataset(crypto):
    twitter_df = pd.read_csv(f'data/processed/twitter/{crypto}_sentiment.csv')
    bit_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_03_05-2022_03_11_minute.csv')
    bit_prices["time"] = pd.to_datetime(bit_prices['time'],unit='s')
    price_dict = bit_prices.set_index("time")["open"].to_dict()
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: pd.to_datetime(x[:-6]))
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: x.replace(second=0))
    twitter_df["future_date"] = twitter_df["created_at"].apply(lambda x: x+timedelta(hours=1))
    twitter_df = twitter_df[twitter_df["future_date"].isin(bit_prices["time"])] ##Filter out dates for which we don't have price data
    twitter_df["price"] = twitter_df["created_at"].apply(lambda x: price_dict[x])
    twitter_df["future_price"] = twitter_df["created_at"].apply(lambda x: price_dict[x+timedelta(hours=1)])
    X = twitter_df[["retweet_count","vader_sentiment_compound","price"]]
    y = twitter_df["future_price"]
    return X,y


In [172]:
X,y = get_dataset("avalanche")

In [173]:
model.fit(X, y, epochs=2000, batch_size=10)

Epoch 1/2000
5/5 [==============================] - 2s 3ms/step - loss: 5541.7676 - mean_squared_error: 5541.7681
Epoch 2/2000
5/5 [==============================] - 0s 4ms/step - loss: 5539.8433 - mean_squared_error: 5539.8433
Epoch 3/2000
5/5 [==============================] - 0s 3ms/step - loss: 5537.9414 - mean_squared_error: 5537.9414
Epoch 4/2000
5/5 [==============================] - 0s 3ms/step - loss: 5536.0493 - mean_squared_error: 5536.0493
Epoch 5/2000
5/5 [==============================] - 0s 3ms/step - loss: 5534.1646 - mean_squared_error: 5534.1646
Epoch 6/2000
5/5 [==============================] - 0s 3ms/step - loss: 5532.2710 - mean_squared_error: 5532.2710
Epoch 7/2000
5/5 [==============================] - 0s 4ms/step - loss: 5530.3682 - mean_squared_error: 5530.3682
Epoch 8/2000
5/5 [==============================] - 0s 3ms/step - loss: 5528.4561 - mean_squared_error: 5528.4561
Epoch 9/2000
5/5 [==============================] - 0s 4ms/step - loss: 5526.5322 - mean

KeyboardInterrupt: 

In [ ]:
X, y = get_dataset("bitcoin")
model.fit(X, y, epochs=2000, batch_size=10)

In [139]:
X, y = get_dataset("doge")
model.fit(X, y, epochs=1000, batch_size=10)

Epoch 1/1000
26/26 [==============================] - 0s 3ms/step - loss: 2.3717e-04 - mean_squared_error: 2.3717e-04
Epoch 2/1000
26/26 [==============================] - 0s 3ms/step - loss: 8.8750e-05 - mean_squared_error: 8.8750e-05
Epoch 3/1000
26/26 [==============================] - 0s 4ms/step - loss: 1.1438e-04 - mean_squared_error: 1.1438e-04
Epoch 4/1000
26/26 [==============================] - 0s 3ms/step - loss: 9.2768e-05 - mean_squared_error: 9.2768e-05
Epoch 5/1000
26/26 [==============================] - 0s 5ms/step - loss: 9.6300e-05 - mean_squared_error: 9.6300e-05
Epoch 6/1000
26/26 [==============================] - 0s 5ms/step - loss: 1.3376e-04 - mean_squared_error: 1.3376e-04
Epoch 7/1000
26/26 [==============================] - 0s 4ms/step - loss: 9.4420e-05 - mean_squared_error: 9.4420e-05
Epoch 8/1000
26/26 [==============================] - 0s 4ms/step - loss: 1.0069e-04 - mean_squared_error: 1.0069e-04
Epoch 9/1000
26/26 [==============================] - 0s

In [140]:
X, y = get_dataset("ethereum")
model.fit(X, y, epochs=1000, batch_size=10)

Epoch 1/1000
35/35 [==============================] - 0s 3ms/step - loss: 4909865.5000 - mean_squared_error: 4909865.5000
Epoch 2/1000
35/35 [==============================] - 0s 5ms/step - loss: 4556524.5000 - mean_squared_error: 4556524.5000
Epoch 3/1000
35/35 [==============================] - 0s 4ms/step - loss: 4554051.5000 - mean_squared_error: 4554051.5000
Epoch 4/1000
35/35 [==============================] - 0s 4ms/step - loss: 4552025.5000 - mean_squared_error: 4552025.5000
Epoch 5/1000
35/35 [==============================] - 0s 4ms/step - loss: 4550260.0000 - mean_squared_error: 4550260.5000
Epoch 6/1000
35/35 [==============================] - 0s 4ms/step - loss: 4548669.0000 - mean_squared_error: 4548669.0000
Epoch 7/1000
35/35 [==============================] - 0s 4ms/step - loss: 4547204.5000 - mean_squared_error: 4547204.5000
Epoch 8/1000
35/35 [==============================] - 0s 4ms/step - loss: 4545838.5000 - mean_squared_error: 4545838.5000
Epoch 9/1000
35/35 [====

In [141]:
X, y = get_dataset("solana")
model.fit(X, y, epochs=1000, batch_size=10)

Epoch 1/1000
6/6 [==============================] - 0s 3ms/step - loss: 452258.6250 - mean_squared_error: 452258.6250
Epoch 2/1000
6/6 [==============================] - 0s 3ms/step - loss: 436677.1562 - mean_squared_error: 436677.1562
Epoch 3/1000
6/6 [==============================] - 0s 3ms/step - loss: 394392.6562 - mean_squared_error: 394392.6562
Epoch 4/1000
6/6 [==============================] - 0s 4ms/step - loss: 310599.6250 - mean_squared_error: 310599.6250
Epoch 5/1000
6/6 [==============================] - 0s 3ms/step - loss: 282378.2188 - mean_squared_error: 282378.2188
Epoch 6/1000
6/6 [==============================] - 0s 5ms/step - loss: 126597.9062 - mean_squared_error: 126597.9062
Epoch 7/1000
6/6 [==============================] - 0s 5ms/step - loss: 83459.2188 - mean_squared_error: 83459.2188
Epoch 8/1000
6/6 [==============================] - 0s 4ms/step - loss: 82659.8594 - mean_squared_error: 82659.8594
Epoch 9/1000
6/6 [==============================] - 0s 4ms/s